<a href="https://colab.research.google.com/github/kazemnejad/tensorflow-2-tutorial/blob/master/part_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>